### 基于物品的协同过滤算法
基于物品的协同过滤算法是目前业界应用最多的算法。
基于用户的协同过滤算法在一些网站中(如Digg)中得到了应用，但该算法有一些缺点。
* 随着用户数目越来越大，计算**用户兴趣相似度矩阵**将越来越困难，其运算时间复杂度和空间复杂度的增长和用户数的增长近似于平方关系。
* 基于用户的协同过滤很难对推荐结果作出解释

基于物品的协同过滤算法（简称ItemCF）给用户推荐那些他们之前喜欢的物品相似的物品。比如，该算法会因为你购买《数据挖掘》而给你推荐《机器学习》。不过,ItemCF算法并**不利用物品的内容属性**计算物品之间的相似度，它主要通过**分析用户的行为记录**计算物品之间的相似度。

该算法认为，物品A和物品B具有很大的相似度是因为喜欢物品A的用户大都喜欢物品B。

基于物品的协同过滤算法主要分为两步。  
（1）计算**物品之间的相似度**  
（2）根据**物品的相似度和用户的历史行为**给用户生成推荐列表。

Wij = (|N(i)&&N(j)|)/|N(i)|
分母|N(i)|是喜欢物品i的用户数，而分子是同时喜欢物品i和物品j的用户数。因此，上述公式可以理解为喜欢物品i的用户中有多少比例的用户也喜欢物品j

上述公式虽然看起来很有道理，但是却存在一个问题。如果物品j很热门，很多人喜欢，那么Wij就会很大，接近1。因此，会造成任何物品都会和热门的物品有很大的相似度，这对于致力于挖掘长尾信息的推荐系统来说显然不是一个好的特征。

为了避免推荐出热门的物品，可以用下面的公式。
Wij = (|N(i)&&N(j)|)/sqrt(|N(i)|* |N(j)|)
这个公式**惩罚了物品j的权重**，因此减轻了热门物品会和很多物品相似的可能性。

从上面的定义可以看到，在协同过滤中**两个物品产生相似度是因为它们共同被很多用户喜欢**，也就是说**每个用户都可以通过它们的历史兴趣列表给物品“贡献”相似度**。

这里面蕴含着一个假设，就是每个用户的兴趣都局限在某几个方面，因此如果两个物品属于一个用户的兴趣列表，那么这两个物品可能就属于有限的几个领域，而如果两个物品属于很多用户的兴趣列表，那么它们就可能属于同一个领域，因而有很大的相似度。

In [1]:
def ItemSimilarity(train):
    # calculate co-rated users between items
    C = dict()
    N = dict()
    for u, items in train.items():
        for i in items:
            N[i] += 1
            for j in items:
                if i == j:
                    continue
                C[i][j] += 1
    
    #calculate finial similarity matrix X
    W = dict()
    for i, related_items in C.items():
        for j, cij in related_items.items():
            W[u][v] = cij / math.sqrt(N[i]*N[j])
    return W

上面程序计算物品相似性的中，对于每个物品集合，我们将里面的物品两两加一，得到一个矩阵。最终将这些矩阵相加得到上面的C矩阵。其中C[i][j]记录了同时喜欢物品i和物品j的用户数。最后，将C矩阵**归一化**可以得到**物品之间的余弦相似度矩阵W**

和用户历史上感兴趣的物品越相似的物品，越有可能在用户的推荐列表中获得比较高的排名。

In [2]:
def Recommendation(train, user_id, W, K):
    rank = dict()
    ru = train[user_id]
    for i, pi in ru.items():
        for j, wj in sorted(W[i].items(), key=itemgetter(1), reverse=True)[0:K]:
            if j in ru:
                continue
            rank[j] += pi * wj
    return rank

ItemCF的一个优势就是可以提供推荐解释，即利用用户历史上喜欢的物品为现在的推荐结果进行解释。

In [1]:
# 带解释的ItemCF算法
def Recommendation(train, user_id, W, K):
    rank = dict()
    ru = train[user_id]
    # 意思是用户对每一本书有一个感兴趣的概率
    for i, pi in ru.items():
        for j, wj in sorted(W[i].items(), key=itemgetter(1), reverse=True)[0:K]:
            if j in ru:
                continue
            rank[j].weight += pi*wj
            rank[j].reason[i] = pi* wj
    return rank

**用户活跃度对物品相似度的影响**  
在协同过滤中两个物品产生相似度是因为它们共同出现在很多用户的兴趣列表中。换句话说，**每个用户的兴趣列表都对物品的相似度产生贡献**。
但是，并不是每个用户的贡献都是相同的。  
举个例子，有个用户开书店的，买了当当网80%的书准备自己卖，那么，他的购物车里包含当当网80%的书。假设当当网100万书，也就是说他买了80万本。从前面对ItemCF的讨论可以看到，这意味着因为存在这么一个用户，有80万本书两两之间就产生了相似度，也就是说，内存里即将诞生一个80万乘80万的稠密矩阵。

另外可以看到，这个用户虽然活跃，但是买这些书并非都出于自身的兴趣，而且这些书覆盖了当当网图书的很多领域，所以这个用户对于他所购买书的两两相似度的贡献应该远远小于一个只买了十几本自己喜欢的书的文学青年。

John S.Breese在论文中提出了一个称为**IUF（Inverse User Frequence），即用户活跃度对数的倒数的参数**，他也认为活跃用户对物品相似度的贡献应该小于不活跃的用户，他提出应该增加IUF参数来修正物品相似度的计算公式

In [7]:
import math
def ItemSimilarity(train):
    #calculate co-rated users between items
    C = dict()
    N = dict()
    for u, items in train.items():
        for i in items:
            N[i] += 1
            for j in items:
                if i == j:
                    continue
                C[i][j] += 1/math.log(1+len(items) * 1.0)
    
    # calculate finial similarity matrix W
    W = dict()
    for i, related_items in C.items():
        for j, cij in related_items.items():
            W[i][j] = cij /math.sqrt(N(i)*N(j))
    return W

**上面的算法记为Item-IUF**

物品相似度的归一化，对ItemCF的相似度矩阵按最大值归一化，可以提高推荐的准确率。  
归一化的好处不仅仅在于**增加推荐的准确度**，还可以**提高推荐的覆盖率和多样性**

#### UserCF算法和ItemCF算法的推荐原理
UserCF给用户推荐那些和他有**共同兴趣爱好的用户喜欢的物品**，而ItemCF给用户推荐那些**和他之前喜欢的物品类似的物品**。从这个算法的原理可以看到，UserCF的推荐结果着重于反映和用户兴趣相似的**小群体的热点**，而ItemCF的推荐结果着重于**维系用户的历史兴趣。**  
换句话说，UserCF的推荐更社会化，反映了用户所在的小型兴趣群体中物品的热门程度，而ItemCF的推荐更加个性化，反映了用户自己的兴趣传承。

